In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [44]:
accelerator

'cu80'

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter, OrderedDict
import re
from copy import deepcopy
import pandas as pd
import numpy as np
from torch.nn.utils.rnn import PackedSequence,pack_padded_sequence
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

flatten = lambda l: [item for sublist in l for item in sublist]

In [2]:
torch.manual_seed(1111)

In [0]:
# 安装 PyDrive 操作库，该操作每个 notebook 只需要执行一次
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 授权登录，仅第一次的时候会鉴权
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [48]:
# 列出根目录的所有文件
# "q" 查询条件教程详见：https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s, mimeType: %s' % (file1['title'], file1['id'], file1["mimeType"]))

title: data, id: 16zfnt8Cnuznzwj2RHGrxvyfyA6iQPRXx, mimeType: application/vnd.google-apps.folder
title: Colab Notebooks, id: 1DDn16_H4dL75WfkYDpWNxRzqJkH6uvsk, mimeType: application/vnd.google-apps.folder
title: 02 - email-templates.png, id: 0BzICsNHMK-GFdnZhNFBoVXdobkhBa3h2eUtwbXY1ek15Sk53, mimeType: image/png
title: ATEC-Alipay, id: 1UyfXNCoPHh73E4jO-pgombXQShfYkzAVgGfSUnU7OAE, mimeType: application/vnd.google-apps.document
title: The Annotated _Attention is All You Need_.ipynb, id: 1zIkaTyyRtMY0qRTuUxe9J10xR3S9J5Xc, mimeType: application/vnd.google.colab
title: Copy of mixture_of_softmaxes.ipynb, id: 1HiJvBimCdWvFP7SoLs9bWNR-JbCQSRsG, mimeType: application/vnd.google.colab
title: Copy of Hello, Colaboratory, id: 0BzICsNHMK-GFcFBrQkV2UXpMVGM, mimeType: application/vnd.google.colab
title: 2017 Calendar, id: 1jw6VChc9SGayYuK5Zmy2PjaiwWM0tzGFrPG9eatsSNw, mimeType: application/vnd.google-apps.spreadsheet
title: Getting started, id: 0BzICsNHMK-GFc3RhcnRlcl9maWxlX2Rhc2hlclYw, mimeType: app

In [49]:
# '目录 id' in parents
file_list = drive.ListFile({'q': "'16zfnt8Cnuznzwj2RHGrxvyfyA6iQPRXx' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s, mimeType: %s' % (file1['title'], file1['id'], file1["mimeType"]))

title: atec_nlp_sim_train.csv, id: 1py3JAFOLOH2lM-lOqYyP_LfqaImlOqL_, mimeType: text/csv


In [0]:
file = drive.CreateFile({'id': "1py3JAFOLOH2lM-lOqYyP_LfqaImlOqL_"}) 
#这里的下载操作只是缓存，不会在你的Google Drive 目录下多下载一个文件
file.GetContentFile('data.csv', "text/csv") 

In [0]:
df = pd.read_csv('data.csv', sep='\t', names=['number', "sen1", "sen2", "label"], skipinitialspace=True)

In [52]:
df.head(2)

,number,sen1,sen2,label
0,1,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1
1,2,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0


In [0]:
MIN_LENGTH = 3
MAX_LENGTH = 40

In [0]:
def normalize_string(s):
    s = re.sub(r"\s+", r" ", s).strip()
    s = re.sub(r"\d+", r"N", s).strip()
    return s

In [55]:
%%time

X1_r = list(map(normalize_string, df.sen1.tolist()))
X2_r = list(map(normalize_string, df.sen2.tolist()))
y_r = df.label.tolist()

for sen1, sen2, y in zip(X1_r, X2_r, y_r):
  if len(sen1) > MAX_LENGTH or len(sen2) > MAX_LENGTH:
    X1_r.remove(sen1)
    X2_r.remove(sen2)
    y_r.remove(y)
    
    X1_r.append(sen1[:MAX_LENGTH])
    X2_r.append(sen2[:MAX_LENGTH])
    y_r.append(y)
    
print(len(X1_r), len(X2_r), len(y_r))
print(X1_r[0], "@@@@", X2_r[0], "@@@@",y_r[0])

39346 39346 39346
﻿怎么更改花呗手机号码 @@@@ 我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号 @@@@ 1
CPU times: user 510 ms, sys: 1 ms, total: 511 ms
Wall time: 515 ms


In [56]:
vocab = list(set(flatten(X1_r + X2_r)))
len(vocab)

1701

In [0]:
source2index = {'<PAD>':0,'<UNK>':1,'<s>':2,'</s>':3}
for vo in vocab:
    if vo not in source2index.keys():
        source2index[vo]=len(source2index)
index2source = {v:k for k,v in source2index.items()}

In [58]:
index2source[source2index["N"]]

'N'

In [0]:
EPOCH=150
BATCH_SIZE = 128
EMBEDDING_SIZE = 128
HIDDEN_SIZE = 256
LR = 0.01
DECODER_LEARNING_RATIO=5.0
RESCHEDULED=False

In [0]:
USE_CUDA = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor

In [61]:
USE_CUDA

True

In [62]:
LongTensor

torch.cuda.LongTensor

In [0]:
#alpha = Variable(torch.Tensor([0.7, 0.3]))
alpha = Variable(FloatTensor([0.75, 0.25]))  # increase loss weights for positive samples
gamma = 2

def one_hot(index, classes):
    size = index.size() + (classes,)
    view = index.size() + (1,)

    mask = FloatTensor(*size).fill_(0)
    index = index.view(*view)
    ones = 1.

    if isinstance(index, Variable):
        ones = Variable(FloatTensor(index.size()).fill_(1))
        mask = Variable(mask, volatile=index.volatile)

    return mask.scatter_(1, index, ones)


class FocalLoss(nn.Module):

    def __init__(self, gamma=0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps

    def forward(self, input, target):
        y = one_hot(target, input.size(-1))
        logit = F.softmax(input, dim=-1)
        logit = logit.clamp(self.eps, 1. - self.eps)   # pt

        loss = -1 * y * torch.log(logit) * alpha # cross entropy
        loss = loss * (1 - logit) ** self.gamma # focal loss

        return loss.sum()

In [0]:
def getBatch(batch_size,train_data):
    random.shuffle(train_data)
    sindex=0
    eindex=batch_size
    while eindex < len(train_data):
        batch = train_data[sindex:eindex]
        temp = eindex
        eindex = eindex+batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch

In [0]:
def prepare_sequence(seq, to_index):
    idxs = list(map(lambda w: to_index[w] if w in to_index.keys() else to_index["<unk>"], seq))
    return LongTensor(idxs)

In [66]:
%%time
X1_p, X2_p = [],[]
ta_p = []

for s1, s2, ta in zip(X1_r, X2_r, y_r):
    X1_p.append(prepare_sequence(s1,source2index).view(1,-1))
    X2_p.append(prepare_sequence(s2,source2index).view(1,-1))
    ta_p.append(ta)
    
data = list(zip(X1_p, X2_p, ta_p))
np.random.shuffle(data)
train_data = data[:30000]
validate_data = data[30000:35000]
test_data = data[35000:]



CPU times: user 3.29 s, sys: 1.29 s, total: 4.58 s
Wall time: 4.59 s


In [67]:
train_data[0]

(
   510   525   863   591   911   843   402  1637  1600  1032  1569
 [torch.cuda.LongTensor of size 1x11 (GPU 0)], 
 
 Columns 0 to 12 
    13  1544   843   432  1252   706   846   762  1210  1059  1013   988  1356
 
 Columns 13 to 23 
   937   723  1399   331   510   525   377   988   937  1127   723
 [torch.cuda.LongTensor of size 1x24 (GPU 0)], 0)

In [0]:
def pad_to_batch(batch,x_to_ix):
    
    sorted_batch =  sorted(batch, key=lambda b:b[0].size(1),reverse=True) # sort by len
    
    x1, x2, y = list(zip(*sorted_batch))
    max_x1 = max([s.size(1) for s in x1])
    max_x2 = max([s.size(1) for s in x2])
    
    x1_p, x2_p, y_p = [],[],[]
    for i in range(len(batch)):
      
        if x1[i].size(1)<max_x1:
            x1_p.append(torch.cat( [ Variable(x1[i]),Variable(LongTensor([x_to_ix['<PAD>']]*(max_x1-x1[i].size(1)))).view(1,-1) ],1))
        else:
            x1_p.append(Variable(x1[i]))
        
        if x2[i].size(1)<max_x2:
            x2_p.append(torch.cat( [ Variable(x2[i]),Variable(LongTensor([x_to_ix['<PAD>']]*(max_x2-x2[i].size(1)))).view(1,-1) ],1))
        else:
            x2_p.append(Variable(x2[i]))
        
    x1_var = torch.cat(x1_p, 0)
    x2_var = torch.cat(x2_p, 0)
    target_var = Variable(LongTensor(y))
    x1_len = [list(map(lambda s: s ==0, t.data)).count(False) for t in x1_var]
    x2_len = [list(map(lambda s: s ==0, t.data)).count(False) for t in x2_var]
    
    return x1_var, x2_var, target_var, x1_len, x2_len

In [0]:
batch = next(getBatch(BATCH_SIZE,train_data))

In [0]:
pbatch = pad_to_batch(batch,source2index)

In [71]:
pbatch[0]

Variable containing:
  988  1009   324  ...     17   434   666
  402  1637  1041  ...      0     0     0
  268   610   436  ...      0     0     0
       ...          ⋱          ...       
 1588   525   723  ...      0     0     0
 1127   723   510  ...      0     0     0
  219    60  1588  ...      0     0     0
[torch.cuda.LongTensor of size 128x40 (GPU 0)]

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score

def train_perf():
  
  print("========== performance on randomly selected 5000 samples from training data")
  samples = random.sample(train_data, 5000)
  sen1, sen2, targets,sen1_lengths, sen2_lengths = pad_to_batch(samples,source2index)
  outputs = model(sen1, sen2, sen1_lengths, sen2_lengths)
  t_predict = np.array(np.argmax(outputs.data, 1))
  t_targets = np.array(targets.data)
  print("Correct: ", np.sum(t_targets == t_predict))
  print("tn, fp, fn, tp \n", confusion_matrix(t_targets, t_predict).ravel())
  print("precsion: ", precision_score(t_predict, t_targets))
  print("recall: ", recall_score(t_predict, t_targets))
  print("f1 score: ", f1_score(t_predict, t_targets))
  
def validate_perf():
  
  print("========== performance on validation data")
  sen1, sen2, targets,sen1_lengths, sen2_lengths = pad_to_batch(validate_data,source2index)
  outputs = model(sen1, sen2, sen1_lengths, sen2_lengths)
  predict = np.array(np.argmax(outputs.data, 1))
  targets = np.array(targets.data)
  print("Correct: ", np.sum(predict == targets))
  print("tn, fp, fn, tp \n", confusion_matrix(targets, predict).ravel())
  print("precsion: ", precision_score(predict, targets))
  print("recall: ", recall_score(predict, targets))
  print("f1 score: ", f1_score(predict, targets))  

In [74]:
np.sum(np.array([i[2] for i in validate_data]) == 1)

1027

In [75]:
np.sum(np.array([i[2] for i in validate_data]) == 0)

3973

In [0]:
class EncoderV(nn.Module):
    def __init__(self, input_size, embedding_size,hidden_size, n_layers=1,bidirec=False):
        super(EncoderV, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.drop = nn.Dropout(0.1)
        
        if bidirec:
            self.n_direction = 2 
            self.gru = nn.GRU(embedding_size, hidden_size, n_layers, batch_first=True,bidirectional=True)
        else:
            self.n_direction = 1
            self.gru = nn.GRU(embedding_size, hidden_size, n_layers, batch_first=True)
    
    def init_hidden(self,inputs):  # input.size(0) = batch_size
        hidden = Variable(torch.zeros(self.n_layers*self.n_direction,inputs.size(0),self.hidden_size))
        return hidden.cuda() if USE_CUDA else hidden
    
    def init_weight(self):
        self.embedding.weight = nn.init.xavier_uniform(self.embedding.weight)
        self.gru.weight_hh_l0 = nn.init.xavier_uniform(self.gru.weight_hh_l0)
        self.gru.weight_ih_l0 = nn.init.xavier_uniform(self.gru.weight_ih_l0)
    
    def forward(self, x, x_len):
        """
        sequence -> sort -> pad and pack ->process using RNN -> unpack ->unsort

        :param x: Variable
        :param x_len: numpy list
        :return:
        """
        
        hidden = self.init_hidden(x)      #### initialise starting state
        
        """sort"""
        x_sort_idx = np.argsort(x_len)[::-1]
        x_unsort_idx = LongTensor(np.argsort(x_sort_idx))
        x_len = np.array(x_len)[x_sort_idx]
        x = x[LongTensor(x_sort_idx.copy())]
      
        embedded = self.embedding(x)
        embedded = self.drop(embedded)
        
        """pack"""
        x_emb_p = torch.nn.utils.rnn.pack_padded_sequence(embedded, x_len, batch_first=True)
                
        """process using RNN"""
        out_pack, ht = self.gru(x_emb_p, hidden)
        
        """unsort: h"""
        ht = torch.transpose(ht, 0, 1)[
            x_unsort_idx]  # (num_layers * num_directions, batch, hidden_size) -> (batch, ...)
        ht = torch.transpose(ht, 0, 1)

        #print("ht", ht.shape)
        if self.n_layers>1:
            if self.n_direction==2:
                ht = ht[-2:]
                return out_pack, torch.cat((ht[0], ht[1]),1)
            else:
                ht = ht[-1]
                return out_pack, ht

     

In [0]:
class Model(nn.Module):
    
    def __init__(self, encoder1, encoder2, hidden_size):
        
        super(Model, self).__init__()
        self.encoder1 = encoder1
        self.drop = nn.Dropout(0.1)
        self.fc1 = nn.Linear(hidden_size*4, hidden_size)
        self.relu = nn. ReLU()
        self.fc2 = nn.Linear(hidden_size, 2)
        
    def init_weight(self):
        
        self.encoder1.init_weight()
        #self.fc1.weight = nn.init.xavier_uniform(self.fc1.weight)
        #self.fc2.weight = nn.init.xavier_uniform(self.fc2.weight)
        
    def forward(self, sen1, sen2, sen1_lengths, sen2_lengths):
             
        outputs_1, hidden_c1 = encoder1(sen1,sen1_lengths)
        outputs_2, hidden_c2 = encoder1(sen2,sen2_lengths)
        
        hidden_c1 = self.drop(hidden_c1)
        hidden_c2 = self.drop(hidden_c2)
        hidden = torch.cat((hidden_c1, hidden_c2), 1).squeeze(1)  # batch * 2hidden
        #hidden = self.drop(hidden)
        out = self.fc1(hidden)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [0]:
encoder1 = EncoderV(len(source2index),EMBEDDING_SIZE,HIDDEN_SIZE,2,True)
model = Model(encoder1, encoder1, HIDDEN_SIZE)
model.init_weight()

if USE_CUDA:
    model = model.cuda()

#criterion = nn.CrossEntropyLoss()
#criterion = FocalLoss(2)
criterion = FocalLoss(2)
optimizer = optim.Adam(model.parameters(),lr=LR)

In [79]:
model.parameters

<bound method Module.parameters of Model(
  (encoder1): EncoderV(
    (embedding): Embedding(1705, 128)
    (drop): Dropout(p=0.1)
    (gru): GRU(128, 256, num_layers=2, batch_first=True, bidirectional=True)
  )
  (drop): Dropout(p=0.1)
  (fc1): Linear(in_features=1024, out_features=256)
  (relu): ReLU()
  (fc2): Linear(in_features=256, out_features=2)
)>

In [80]:
for i,p in enumerate(model.parameters()):
    if p.requires_grad:
        print(i, ":", p.shape)

0 : torch.Size([1705, 128])
1 : torch.Size([768, 128])
2 : torch.Size([768, 256])
3 : torch.Size([768])
4 : torch.Size([768])
5 : torch.Size([768, 128])
6 : torch.Size([768, 256])
7 : torch.Size([768])
8 : torch.Size([768])
9 : torch.Size([768, 512])
10 : torch.Size([768, 256])
11 : torch.Size([768])
12 : torch.Size([768])
13 : torch.Size([768, 512])
14 : torch.Size([768, 256])
15 : torch.Size([768])
16 : torch.Size([768])
17 : torch.Size([256, 1024])
18 : torch.Size([256])
19 : torch.Size([2, 256])
20 : torch.Size([2])


In [81]:
list(model.parameters())[17]

Parameter containing:
-1.4277e-02 -1.4485e-02 -2.2928e-02  ...   1.6407e-02 -1.4513e-02 -2.7617e-02
 4.0419e-03 -1.3952e-02 -2.2086e-02  ...  -1.3849e-02  2.3486e-02  1.1785e-02
 1.1941e-02 -3.0250e-02  1.3061e-02  ...  -2.3700e-02  1.4854e-02  1.3456e-02
                ...                   ⋱                   ...                
-1.8124e-02 -9.7955e-04 -3.1669e-03  ...  -6.2509e-03 -9.4317e-03  2.3574e-02
 2.5391e-02  2.2048e-02  2.6482e-02  ...  -2.7667e-02 -3.6992e-03 -2.4014e-02
-2.7981e-03 -9.2835e-03  2.5175e-02  ...   1.7646e-02 -2.9831e-02  2.4577e-02
[torch.cuda.FloatTensor of size 256x1024 (GPU 0)]

In [82]:
list(model.parameters())[5]

Parameter containing:
 3.0355e-03 -1.2961e-02  6.0064e-02  ...  -4.5771e-02  3.8890e-02 -2.4952e-02
 3.7672e-02  5.5773e-02 -2.6081e-02  ...   2.9321e-02 -3.2315e-02  4.4971e-03
 1.4381e-02  2.3699e-03 -3.3544e-02  ...  -4.8572e-02 -5.5734e-02 -3.7700e-02
                ...                   ⋱                   ...                
 3.0168e-02 -1.1434e-02 -2.9736e-02  ...  -3.2610e-02  5.6947e-02  4.0127e-02
 3.2363e-02  4.6382e-02 -2.7200e-02  ...   4.1018e-02  2.3845e-02  4.2604e-02
 4.2939e-02  3.3985e-02 -4.3579e-02  ...   3.8251e-02  1.0417e-02  4.2799e-02
[torch.cuda.FloatTensor of size 768x128 (GPU 0)]

In [83]:
### test 
for i,batch in enumerate(getBatch(BATCH_SIZE,train_data)):
    
    # sen1 = batch * l
    sen1, sen2, targets,sen1_lengths, sen2_lengths = pad_to_batch(batch,source2index)
    print("sen1 size: ", sen1.shape)
    #input_masks = torch.cat([Variable(ByteTensor(tuple(map(lambda s: s ==0, t.data)))) for t in sen1]).view(sen1.size(0),-1)
    #model.zero_grad()
    
    # Forward + Backward + Optimize
    optimizer.zero_grad()
    outputs = model(sen1, sen2, sen1_lengths, sen2_lengths)
    print('outputs', outputs.shape)
    print('targets', targets.shape)
    loss = criterion(outputs, targets)
    print('loss', loss)
    loss.backward()
    optimizer.step()
    
    break
    
train_perf()
validate_perf()

sen1 size:  torch.Size([128, 40])
outputs torch.Size([128, 2])
targets torch.Size([128])
loss Variable containing:
 15.1438
[torch.cuda.FloatTensor of size 1 (GPU 0)]

========== performance on randomly selected 5000 samples from training data
Correct:  3916
tn, fp, fn, tp 
 [3916    0 1084    0]
precsion:  0.0
recall:  0.0
f1 score:  0.0
========== performance on validation data


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


Correct:  3973
tn, fp, fn, tp 
 [3973    0 1027    0]
precsion:  0.0
recall:  0.0
f1 score:  0.0


In [84]:
## TN FP
## FN TP

total_step = round(len(train_data) / BATCH_SIZE)

for epoch in range(EPOCH):
    
    total_loss = []
    
    for i,batch in enumerate(getBatch(BATCH_SIZE,train_data)):
        sen1, sen2, targets,sen1_lengths, sen2_lengths = pad_to_batch(batch,source2index)
        #model.zero_grad()
        optimizer.zero_grad()

        # Forward + Backward + Optimize
        outputs = model(sen1, sen2, sen1_lengths, sen2_lengths)
        loss = criterion(outputs, targets)
        total_loss.append(loss.data[0])
        loss.backward()
        
        
        torch.nn.utils.clip_grad_norm(model.parameters(), 0.5) # gradient clipping
        optimizer.step()

        if (i+1) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Mean Loss: {:.4f}' 
                       .format(epoch+1, EPOCH, i+1, total_step, np.mean(total_loss)))
            total_loss = []
    
    if (epoch + 1) %10 == 0:
        train_perf()
        validate_perf()
    
    #if RESCHEDULED == False and epoch  == EPOCH//4:
    if (epoch + 1) %25 == 0:
        LR *= 0.1
        print("LR now is: ", LR)
        optimizer = optim.Adam(model.parameters(), lr=LR)
        #RESCHEDULED = True

Epoch [1/150], Step [20/234], Mean Loss: 60.9731
Epoch [1/150], Step [40/234], Mean Loss: 16.6609
Epoch [1/150], Step [60/234], Mean Loss: 7.5086
Epoch [1/150], Step [80/234], Mean Loss: 32.5770
Epoch [1/150], Step [100/234], Mean Loss: 58.2470
Epoch [1/150], Step [120/234], Mean Loss: 8.4096
Epoch [1/150], Step [140/234], Mean Loss: 7.0080
Epoch [1/150], Step [160/234], Mean Loss: 6.9316
Epoch [1/150], Step [180/234], Mean Loss: 6.5598
Epoch [1/150], Step [200/234], Mean Loss: 7.4286
Epoch [1/150], Step [220/234], Mean Loss: 7.2922
Epoch [2/150], Step [20/234], Mean Loss: 6.9981
Epoch [2/150], Step [40/234], Mean Loss: 7.1628
Epoch [2/150], Step [60/234], Mean Loss: 6.8713
Epoch [2/150], Step [80/234], Mean Loss: 6.7820
Epoch [2/150], Step [100/234], Mean Loss: 6.9583
Epoch [2/150], Step [120/234], Mean Loss: 6.8134
Epoch [2/150], Step [140/234], Mean Loss: 6.7407
Epoch [2/150], Step [160/234], Mean Loss: 6.7690
Epoch [2/150], Step [180/234], Mean Loss: 6.5154
Epoch [2/150], Step [200

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


Correct:  3973
tn, fp, fn, tp 
 [3973    0 1027    0]
precsion:  0.0
recall:  0.0
f1 score:  0.0
Epoch [11/150], Step [20/234], Mean Loss: 6.6734
Epoch [11/150], Step [40/234], Mean Loss: 7.1429
Epoch [11/150], Step [60/234], Mean Loss: 7.2135
Epoch [11/150], Step [80/234], Mean Loss: 6.5440
Epoch [11/150], Step [100/234], Mean Loss: 6.7476
Epoch [11/150], Step [120/234], Mean Loss: 6.6975
Epoch [11/150], Step [140/234], Mean Loss: 7.1095
Epoch [11/150], Step [160/234], Mean Loss: 6.7437
Epoch [11/150], Step [180/234], Mean Loss: 6.8401
Epoch [11/150], Step [200/234], Mean Loss: 6.6917
Epoch [11/150], Step [220/234], Mean Loss: 6.7649
Epoch [12/150], Step [20/234], Mean Loss: 6.6881
Epoch [12/150], Step [40/234], Mean Loss: 6.9102
Epoch [12/150], Step [60/234], Mean Loss: 7.0092
Epoch [12/150], Step [80/234], Mean Loss: 6.5706
Epoch [12/150], Step [100/234], Mean Loss: 6.4731
Epoch [12/150], Step [120/234], Mean Loss: 6.6012
Epoch [12/150], Step [140/234], Mean Loss: 6.5597
Epoch [12/1

KeyboardInterrupt: ignored

In [0]:
torch.save(model.state_dict(), 'rnn-based.ckpt')